In [1]:
# imports
import json
import re
from song_scorer import song_scorer
from string_utils import str_utils

In [2]:
# define testing functions
class dummy_log():
    def log(self, inp):
        print(f"- DUMMY LOG: {inp}")

def test_match(sp_track_data, local_track_data) -> tuple[dict, bool]:
    scorer = song_scorer(sp_track_data, dummy_log())
    matches = scorer.check_song(local_track_data)
    return matches, not scorer.not_mininmum_matched(matches)

def splitter(inp: str) -> list[str]:
    base = re.split('[^a-zA-Z0-9]', inp)
    filt = []
    for word in base:
        if len(word) == 0:
            continue
        filt.append(word)
    return filt

def find_potential_sp_tracks(track_name:str, playlist_key:str, sp_tracks: list[dict]) -> list[dict]:
    max_match_tracks = None
    max_match = -1
    split_track_name = splitter(track_name)
    for sp_track in sp_tracks[playlist_key]:
        split_name = splitter(sp_track["name"])
        cur_cnt = 0
        for key_word in split_track_name:
            if key_word in split_name:
                cur_cnt += 1
        if cur_cnt > max_match:
            max_match_tracks = [sp_track]
            max_match = cur_cnt 
        elif cur_cnt == max_match:
            max_match_tracks.append(sp_track)
    return max_match_tracks

In [3]:
# load data

with open("cache/sp_tracks.json", "r") as f:
    sp_tracks = json.load(f)
with open("cache/all_converted_tracks.json", "r") as f:
    raw_local_tracks: dict = json.load(f)
local_tracks = {}
for pth, local_track in raw_local_tracks.items():
    local_tracks[pth.lower()] = local_track

In [4]:
track_name = "Father and son"
playlist_key = "Halls Gap"
path_to_local = r"d:\Music_CONVERTED\Cat Stevens\{1970} Tea for the Tillerman [16-44.1]\Cat Stevens - 10 - Father & Son.mp3"

possible_sp_tracks = find_potential_sp_tracks(track_name, playlist_key, sp_tracks)
sp_track = possible_sp_tracks[0]

local_track = local_tracks[path_to_local.lower()]

print(f"-- Spotify track data:\n{json.dumps(sp_track, indent=2)}\n")
print(f"-- Local track data:\n{json.dumps(local_track,  indent=2)}\n")

-- Spotify track data:
{
  "name": "Father And Son",
  "album": "Tea For The Tillerman (Remastered 2020)",
  "artists": [
    "Yusuf / Cat Stevens"
  ],
  "duration_ms": 221120,
  "tracknumber": 10,
  "added_at": "2025-06-26T14:09:46Z",
  "added_at_epoch": 1750912786.0
}

-- Local track data:
{
  "artists": [
    "cat stevens"
  ],
  "name": "father & son",
  "duration_ms": 221533,
  "album": "tea for the tillerman",
  "tracknumber": "10",
  "file_path": "D:\\Music_CONVERTED\\Cat Stevens\\{1970} Tea for the Tillerman [16-44.1]\\Cat Stevens - 10 - Father & Son.mp3",
  "modified_time": 1681719627.318985
}



In [5]:
matches, matched = test_match(sp_track, local_track)
print(f"Matched: {matched}\nMatches: {json.dumps(matches, indent=2)}")

- DUMMY LOG: Split Artist Matched: local "{'cat stevens'}", spotify "['yusuf / cat stevens']" 
Matched: True
Matches: {
  "artists": 0.5789473684210527,
  "name": 0.8571428571428571,
  "duration_ms": 0.7967759856709842,
  "album": 1.0,
  "tracknumber": 0
}
